# Examine filtered data
For trying to improve quality of filtering

In [ ]:
# Load filtered data (CSVs)
import pandas as pd
import os
from tqdm.notebook import tqdm

filtered_dirpath = '/storage2/carley02/mamille3/adl_covid/output/processed_tweets_csv'
search_matches = []
for fname in tqdm(os.listdir(filtered_dirpath)):
    fpath = os.path.join(filtered_dirpath, fname)
    data = pd.read_csv(fpath)
    search_matches = data.search_match.value_counts()

In [2]:
import pandas as pd

test = pd.DataFrame({'a': 1, 'b': 2})
test

ValueError: If using all scalar values, you must pass an index

# Load, filter COVID Twitter data
By keywords and media attachments

In [6]:
import re

terms_path = '../resources/antisemitic_terms.txt'
with open(terms_path) as f:
    search_terms = f.read().splitlines()
pats = [re.compile(r'\b{}\b'.format(re.escape(term.lower()))) for term in search_terms]

import pdb

def select_tweet(tweet):
    """ See if a tweet is worth keeping (matches enough criteria) """
    select = False
    
    # Basic cleaning
    if len(tweet) == 1 and 'limit' in tweet:
        return select
    
    # Language is English
    if tweet['lang'] != 'en':
        return select
    
    # Has media
    if 'media' not in tweet['entities']:
        return select
    
    # Contains possibly antisemitic terms
    if 'extended_tweet' in tweet:
        text = tweet['extended_tweet']['full_text'].lower()
    else:
        text = tweet['text'].lower()
    for p in pats:
        m = re.search(p, text)
        # if any([re.search(p, tweet['extended_tweet']['full_text'].lower()) for p in pats]):
        if m is not None:
            select = m.group()
            # tqdm.write('one selected')
            
    return select

# Load COVID Twitter data (Carley lab)
import os
import gzip
import json
from tqdm.notebook import tqdm

basepath = '/storage3/coronavirus/'

# Older data
dirname = 'json_keyword_stream'
# Newer data
dirname = 'json_keyword_stream_mike'
selected = []
n_selected = 0
for fname in sorted([fname for fname in os.listdir(os.path.join(basepath, dirname)) if fname.endswith('.json.gz')])[:1]:
    print(fname)
    fpath = os.path.join(basepath, dirname, fname)
    with gzip.open(fpath, 'rb') as f:
        for i, line in tqdm(enumerate(f), total=9560357):
            if len(line) == 1:
                continue
        # for i, line in tqdm(enumerate(f), total=974483, bar_format='selected: {postfix} | Elapsed: {elapsed} | {rate_fmt}', postfix=n_selected):
            tweet = json.loads(line)
            match = select_tweet(tweet)
            if match:
                tweet['search_match'] = match
                selected.append(tweet)
                n_selected += 1
            # if i > 100:
            #     break
n_selected

covid_20200602.json.gz


  0%|          | 0/974483 [00:00<?, ?it/s]

31

In [11]:
pat = re.compile(r'\b88\b', re.UNICODE)
print(re.search(pat, 'https://t.co/FC88v4dWnc'))

None


In [7]:
n_selected

31

In [4]:
from collections import Counter

Counter([select['search_match'] for select in selected]).most_common()

[('israel', 26)]

In [8]:
# Save out selected
outpath = os.path.join('../output', f'{fname.split(".")[0]}.jsonl')
with open(outpath, 'w') as f:
    f.write('\n'.join([json.dumps(tweet) for tweet in selected]))

In [13]:
no_media = [select for select in selected if 'media' not in select['entities']]
len(no_media)

648

In [15]:
no_media[0]['entities']

{'hashtags': [],
 'urls': [{'url': 'https://t.co/7oecmaPjY2',
   'expanded_url': 'https://twitter.com/i/web/status/1222538649788649472',
   'display_url': 'twitter.com/i/web/status/1…',
   'indices': [117, 140]}],
 'user_mentions': [{'screen_name': 'ScottAdamsSays',
   'name': 'Scott Adams',
   'id': 2853461537,
   'id_str': '2853461537',
   'indices': [6, 21]}],
 'symbols': []}

In [23]:
test_tweet = {'lang': 'en', 'text': 'goyim'}
select_tweet(test_tweet)

one selected


True

## Process selected output tweets to CSV

In [2]:
# Load filtered tweets
import os
import json

import pandas as pd
from tqdm.notebook import tqdm

dirpath = '../output/tweets_json/'
for fname in tqdm(sorted(os.listdir(dirpath))[:1]):
    fpath = os.path.join(dirpath, fname)
    with open(fpath) as f:
        data = pd.json_normalize([json.loads(line) for line in f.read().splitlines()])
sorted(data.columns.tolist())

  0%|          | 0/1 [00:00<?, ?it/s]

['created_at',
 'id',
 'id_str',
 'text',
 'display_text_range',
 'source',
 'truncated',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'quote_count',
 'reply_count',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'filter_level',
 'lang',
 'timestamp_ms',
 'search_match',
 'user.id',
 'user.id_str',
 'user.name',
 'user.screen_name',
 'user.location',
 'user.url',
 'user.description',
 'user.translator_type',
 'user.protected',
 'user.verified',
 'user.followers_count',
 'user.friends_count',
 'user.listed_count',
 'user.favourites_count',
 'user.statuses_count',
 'user.created_at',
 'user.utc_offset',
 'user.time_zone',
 'user.geo_enabled',
 'user.lang',
 'user.contributors_enabled',
 'user.is_translator',
 'user.profile_background_color',
 'user.profile_background_image_url',
 'user.

In [20]:
sorted(data.columns.tolist())

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities.hashtags',
 'entities.media',
 'entities.symbols',
 'entities.urls',
 'entities.user_mentions',
 'extended_entities.media',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status.contributors',
 'quoted_status.coordinates',
 'quoted_status.created_at',
 'quoted_status.display_text_range',
 'quoted_status.entities.hashtags',
 'quoted_status.entities.symbols',
 'quoted_status.entities.urls',
 'quoted_status.entities.user_mentions',
 'quoted_status.extended_tweet.display_text_range',
 'quoted_status.extended_tweet.entities.hashtags',
 'quoted_status.extended_tweet.entities.media',
 'quoted_status.extended_tweet.entities.symbols',
 'quoted_status.extended_tweet.e

In [25]:
data[['user.screen_name', 'id_str', 'text', 'source']].sample(10)

,user.screen_name,id_str,text,source
10,mufcnews2019,1267799776008929280,Israel's OurCrowd to raise $100 million for coronavirus technologies https://t.co/WPKpBBL1rY https://t.co/VtMLf8zShL,"<a href=""https://dlvrit.com/"" rel=""nofollow"">dlvr.it</a>"
8,OttawaCitizen,1267789447782961152,"City camps cancelled due to COVID-19 pandemic, replacement program uncertain https://t.co/9eyKH0BPni https://t.co/M2IHmFLYX1","<a href=""https://www.echobox.com"" rel=""nofollow"">Echobox</a>"
17,OnlyAprilMoreno,1267854528336060417,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>"
25,Aaisxaaa,1267933876225269760,RT @itvnews: Gaza's garment industry boosted by coronavirus and unlikely demand from Israel https://t.co/lVQf3juR7m https://t.co/e0aeS5fhvo,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>"
5,DalalIRIQAT,1267753699486695424,My most recent publications #Trump #dealofthecentury #Palestine #covid19 #Annexation #Israel #Occupation https://t.co/64ov41mVEI,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>"
11,Star11981587,1267823889670782978,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>"
19,ChoiceMediatv,1267871149796601858,Coronavirus Aid for Jewish Day Schools Caught Up in DeVos Debate\n\nby @MollyBoigon\n\nhttps://t.co/xVxuY92g7j https://t.co/wWNneUniBD,"<a href=""https://buffer.com"" rel=""nofollow"">Buffer</a>"
16,josefapintoo1,1267847529745563650,#Anonymous #Chile #PizzaGate #COVID19 #junio2020 #DonaldTrump #Area51 #coronavirus \n\nLINK:https://t.co/GGEbCigDRm https://t.co/a8nSCrE2vR,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>"
15,kinghills42,1267842168129032192,Rapture is almost here 🇮🇱🇮🇱\n#Jewish \n#Israel \n#BlackLivesMatter \n#policebrutalityke \n#WarAgainstVirus \n#COVID19 https://t.co/dcBHQbRNjx,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>"
1,ArabyOrg,1267726000907059212,"With children infected by coronavirus, Israel fears second wave https://t.co/jnXqq1UNxZ via @almonitor https://t.co/Dwolt1rMKH","<a href=""https://www.tweetsuite.com/"" rel=""nofollow"">Tweet Suite</a>"


In [19]:
pd.set_option('display.max_colwidth', None)
data.loc[0, ['text', 'retweeted_status.id_str']]

text                       Coronavirus cases in Israel increased once public transportation restrictions were loosened https://t.co/klKN4ZXBhA https://t.co/4PqJe9EDp0
retweeted_status.id_str                                                                                                                                            NaN
Name: 0, dtype: object

In [29]:
# Add URL column (just for demonstration purposes, not for actual coding)
# Not necessary: Twitter does this as the last word in the text
data['url'] = [f'https://twitter.com/{screen_name}/status/{status_id_str}' for screen_name, status_id_str in zip(data['user.screen_name'], data['id_str'])]
data[['user.screen_name', 'id_str', 'url']].sample(5)

,user.screen_name,id_str,url
13,RommyPichiling3,1267834967859630085,https://twitter.com/RommyPichiling3/status/1267834967859630085
1,ArabyOrg,1267726000907059212,https://twitter.com/ArabyOrg/status/1267726000907059212
11,Star11981587,1267823889670782978,https://twitter.com/Star11981587/status/1267823889670782978
23,bvbyalice,1267910327284441088,https://twitter.com/bvbyalice/status/1267910327284441088
10,mufcnews2019,1267799776008929280,https://twitter.com/mufcnews2019/status/1267799776008929280


In [28]:
# Select columns to keep
"""
I think we need all data related to the tweet content for the coding:
- tweet text
- hashtags and @'s
- links or attachment info (if there is one or more)

Additional meta information might be useful I guess:
- timestamp
- number of likes, RTs (Quote RTs?), replies
"""

selected_cols = ['created_at', 'id_str', 'text', 'in_reply_to_status_id_str', 'geo', 'coordinates', 'place', 
                 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'possibly_sensitive',
                 'search_match',
                 'user.id_str', 'user.location', 'user.geo_enabled',
                 'entities.hashtags', 'entities.urls', 'entities.user_mentions', 'entities.symbols', 'entities.media', 'extended_entities.media',
                 'retweeted_status.id_str', 'retweeted_status.user.location', 'retweeted_status.geo', 'retweeted_status.coordinates', 'retweeted_status.place', 
                 'retweeted_status.quote_count', 'retweeted_status.reply_count', 'retweeted_status.retweet_count',
                 'quoted_status.id_str', 'quoted_status.user.location', 'quoted_status.geo', 'quoted_status.coordinates', 'quoted_status.place', 
                 'quoted_status.quote_count', 'quoted_status.reply_count', 'quoted_status.retweet_count',
                ]
selected_data = data[selected_cols]
selected_data.head()

,created_at,id_str,text,in_reply_to_status_id_str,geo,coordinates,place,is_quote_status,quote_count,reply_count,...,retweeted_status.reply_count,retweeted_status.retweet_count,quoted_status.id_str,quoted_status.user.location,quoted_status.geo,quoted_status.coordinates,quoted_status.place,quoted_status.quote_count,quoted_status.reply_count,quoted_status.retweet_count
0,Tue Jun 02 07:04:37 +0000 2020,1267713717862612993,Coronavirus cases in Israel increased once public transportation restrictions were loosened https://t.co/klKN4ZXBhA https://t.co/4PqJe9EDp0,None,None,None,None,False,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tue Jun 02 07:53:26 +0000 2020,1267726000907059212,"With children infected by coronavirus, Israel fears second wave https://t.co/jnXqq1UNxZ via @almonitor https://t.co/Dwolt1rMKH",None,None,None,None,False,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tue Jun 02 08:16:33 +0000 2020,1267731819836796929,Study: Israel doesn't have coronavirus herd immunity https://t.co/ylH13ipKOP via ArutzSheva_En https://t.co/AI2s5NzqKc,None,None,None,None,False,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tue Jun 02 08:32:58 +0000 2020,1267735950181969921,"City camps cancelled because of pandemic, replacement program uncertain https://t.co/mANlby5C88 https://t.co/E6UkOaoaST",None,None,None,None,False,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tue Jun 02 08:33:36 +0000 2020,1267736110257639424,"RT @ottawasuncom: City camps cancelled because of pandemic, replacement program uncertain https://t.co/mANlby5C88 https://t.co/E6UkOaoaST",None,None,None,None,False,0,0,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# De-identify
""" Remove @mentions """


In [36]:
data[data['entities.user_mentions'].map(lambda x: len(x) > 0)][['text', 'entities.user_mentions']]

,text,entities.user_mentions
1,"With children infected by coronavirus, Israel fears second wave https://t.co/jnXqq1UNxZ via @almonitor https://t.co/Dwolt1rMKH","[{'screen_name': 'AlMonitor', 'name': 'Al-Monitor', 'id': 290061912, 'id_str': '290061912', 'indices': [92, 102]}]"
4,"RT @ottawasuncom: City camps cancelled because of pandemic, replacement program uncertain https://t.co/mANlby5C88 https://t.co/E6UkOaoaST","[{'screen_name': 'ottawasuncom', 'name': 'Ottawa Sun', 'id': 27646245, 'id_str': '27646245', 'indices': [3, 16]}]"
11,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"[{'screen_name': 'InM0re', 'name': 'm0re in quarentine 🏳️‍🌈', 'id': 1261296928714887170, 'id_str': '1261296928714887170', 'indices': [3, 10]}]"
12,"RT @ottawasuncom: City camps cancelled because of pandemic, replacement program uncertain https://t.co/hhMI4rPp5X https://t.co/LiJWv2zBOz","[{'screen_name': 'ottawasuncom', 'name': 'Ottawa Sun', 'id': 27646245, 'id_str': '27646245', 'indices': [3, 16]}]"
13,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"[{'screen_name': 'InM0re', 'name': 'm0re in quarentine 🏳️‍🌈', 'id': 1261296928714887170, 'id_str': '1261296928714887170', 'indices': [3, 10]}]"
17,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"[{'screen_name': 'InM0re', 'name': 'm0re in quarentine 🏳️‍🌈', 'id': 1261296928714887170, 'id_str': '1261296928714887170', 'indices': [3, 10]}]"
18,RT @ZKahronicNebula: False flags to create a Race War because everybody knows covid19 is a hoax too! https://t.co/4tEQESO0gc,"[{'screen_name': 'ZKahronicNebula', 'name': 'Mr. Wake The Fuck Up', 'id': 745669328, 'id_str': '745669328', 'indices': [3, 19]}]"
19,Coronavirus Aid for Jewish Day Schools Caught Up in DeVos Debate\n\nby @MollyBoigon\n\nhttps://t.co/xVxuY92g7j https://t.co/wWNneUniBD,"[{'screen_name': 'MollyBoigon', 'name': 'Molly Boigon', 'id': 2457422802, 'id_str': '2457422802', 'indices': [69, 81]}]"
20,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,"[{'screen_name': 'InM0re', 'name': 'm0re in quarentine 🏳️‍🌈', 'id': 1261296928714887170, 'id_str': '1261296928714887170', 'indices': [3, 10]}]"
22,RT @itvnews: Gaza's garment industry boosted by coronavirus and unlikely demand from Israel https://t.co/lVQf3juR7m https://t.co/e0aeS5fhvo,"[{'screen_name': 'itvnews', 'name': 'ITV News', 'id': 21866939, 'id_str': '21866939', 'indices': [3, 11]}]"


In [44]:
import re

def remove_mentions(text, user_mentions):
    """ Remove mentions from a text """
    new_text = text
    usernames = [mention['screen_name'] for mention in user_mentions]
    for username in usernames:
        new_text = re.sub(username, 'USER', new_text, flags=re.IGNORECASE)
    return new_text

def process_text(text, user_mentions):
    new_text = remove_mentions(text, user_mentions)
    return ' '.join(new_text.split(' ')[:-1])

In [45]:
selected_data['processed_text'] = [process_text(text, mentions) for text, mentions in zip(selected_data['text'], selected_data['entities.user_mentions'])]
selected_data[['text', 'processed_text']].sample(10)

/tmp/ipykernel_6126/307755961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['processed_text'] = [process_text(text, mentions) for text, mentions in zip(selected_data['text'], selected_data['entities.user_mentions'])]


,text,processed_text
21,Gaza's garment industry boosted by coronavirus and unlikely demand from Israel https://t.co/lVQf3juR7m https://t.co/e0aeS5fhvo,Gaza's garment industry boosted by coronavirus and unlikely demand from Israel https://t.co/lVQf3juR7m
6,If the President could stop George Soros paying people to be anarchists it might help https://t.co/GY6ZsYzvCR,If the President could stop George Soros paying people to be anarchists it might help
10,Israel's OurCrowd to raise $100 million for coronavirus technologies https://t.co/WPKpBBL1rY https://t.co/VtMLf8zShL,Israel's OurCrowd to raise $100 million for coronavirus technologies https://t.co/WPKpBBL1rY
23,This year got us like dis frfr\n#Anonymuos #Anonymousisback #PizzaGate #coronavirus #worldwar3 #OpDeathEaters https://t.co/URnKeQikOf,This year got us like dis frfr\n#Anonymuos #Anonymousisback #PizzaGate #coronavirus #worldwar3 #OpDeathEaters
11,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,RT @USER: #anonymus he is back #PizzaGate #coronavirus
15,Rapture is almost here 🇮🇱🇮🇱\n#Jewish \n#Israel \n#BlackLivesMatter \n#policebrutalityke \n#WarAgainstVirus \n#COVID19 https://t.co/dcBHQbRNjx,Rapture is almost here 🇮🇱🇮🇱\n#Jewish \n#Israel \n#BlackLivesMatter \n#policebrutalityke \n#WarAgainstVirus \n#COVID19
13,RT @InM0re: #anonymus he is back #PizzaGate #coronavirus https://t.co/SpPItfnkog,RT @USER: #anonymus he is back #PizzaGate #coronavirus
7,Study: Israel 'Far From Herd Immunity' to Coronavirus - https://t.co/EFx1xSrrkV #jewish #israel #news #judaism https://t.co/NARqy1fYfV,Study: Israel 'Far From Herd Immunity' to Coronavirus - https://t.co/EFx1xSrrkV #jewish #israel #news #judaism
5,My most recent publications #Trump #dealofthecentury #Palestine #covid19 #Annexation #Israel #Occupation https://t.co/64ov41mVEI,My most recent publications #Trump #dealofthecentury #Palestine #covid19 #Annexation #Israel #Occupation
12,"RT @ottawasuncom: City camps cancelled because of pandemic, replacement program uncertain https://t.co/hhMI4rPp5X https://t.co/LiJWv2zBOz","RT @USER: City camps cancelled because of pandemic, replacement program uncertain https://t.co/hhMI4rPp5X"


In [40]:
selected_data.text.sample(20)

9                            Israel's OurCrowd to raise US$100 million for coronavirus technologies https://t.co/MN6jhEZeEH https://t.co/K1IlHQHSEU
3                           City camps cancelled because of pandemic, replacement program uncertain https://t.co/mANlby5C88 https://t.co/E6UkOaoaST
0       Coronavirus cases in Israel increased once public transportation restrictions were loosened https://t.co/klKN4ZXBhA https://t.co/4PqJe9EDp0
5                  My most recent publications #Trump #dealofthecentury #Palestine #covid19 #Annexation #Israel #Occupation https://t.co/64ov41mVEI
26                     Coronavirus Backlash May Shut Schools Again - https://t.co/utdVk7zGEW #jewish #israel #news #judaism https://t.co/EFFBZXBIP6
15    Rapture is almost here 🇮🇱🇮🇱\n#Jewish \n#Israel \n#BlackLivesMatter \n#policebrutalityke \n#WarAgainstVirus \n#COVID19 https://t.co/dcBHQbRNjx
20                                                                 RT @InM0re: #anonymus he is back #PizzaGate #

In [43]:
selected_data['url'] = [el[-1] for el in selected_data.text.str.split(' ')]
selected_data['url'].head()

/tmp/ipykernel_6126/3069831366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['url'] = [el[-1] for el in selected_data.text.str.split(' ')]


0    https://t.co/4PqJe9EDp0
1    https://t.co/Dwolt1rMKH
2    https://t.co/AI2s5NzqKc
3    https://t.co/E6UkOaoaST
4    https://t.co/E6UkOaoaST
Name: url, dtype: object

In [14]:
outpath = os.path.join(dirpath, 'filtered_covid_tweets.csv')
out_df.to_csv(outpath)